In [1]:
import warnings
warnings.simplefilter('ignore')

import re
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm import tqdm

In [2]:
fr1 = pd.read_excel('data_original/finereader/pzz_1_8279.xlsx', header=None)
fr2 = pd.read_excel('data_original/finereader/pzz_8279_end.xlsx', header=None)

In [3]:
fr1.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Крупенина О.Н.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""Защитных зонах"" есть ГПЗУ. Считаю, что проект...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
fr2.head(3)

,0,1,2,3,4,5,6
0,Александр Иванович,NaN,NaN,NaN,NaN,NaN,NaN
1,Мокеева Г.Л.,Я согласен с ПЗЗ чтобы жилательно снести хруще...,Принято к сведению.,NaN,NaN,NaN,NaN
2,Марей ев Валерий Анатольевич,Я согласен с проектом ПЗЗ Больше бесплатных па...,"Принято к сведению. Предложения/замечания, отн...",NaN,NaN,NaN,NaN


In [5]:
fr1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
count,199291,37725,33630,584,192,90,46,26,23,10,16,1,13,18,2
unique,117914,18679,4984,353,82,47,24,19,22,10,16,1,13,18,2
top,Обоснование:,"Я , житель Юго-Западного округа Москвы, выступ...",Принято к сведению,к,предмету,публичных,публичных,номером,Объекты учебно воспитательного назначения,пред ►•ость по сущ. ик>,15,застро тыс м'/га,рассмотрению Архитекту рным советом города Мо,№«шп>оет»о«'м««проекте ПЗЗ для территориальной...,"проекте 1133 для территориальной зоны, в котор..."
freq,918,1874,12545,20,20,20,16,8,2,1,1,1,1,1,1


In [6]:
fr2.describe()

,0,1,2,3,4,5,6
count,65081,27893,27051,193,36,4,2
unique,44027,9455,1031,64,23,4,2
top,Замечание не рекомендовано к учету. В проекте ...,"Да, поддерживаю",Принято к сведению.,Принято к сведению.,Принято к сведению.,Одобрение ППТ,от 15.12.15 №888-1111
freq,160,4382,11628,125,11,1,1


Видим кучу левых столбцов, будем удалять

### Извлечём номера страниц из текста

Идея: по каждой строчке складываем значения по всем столбцам, если результат - число - то это номер страницы. Но нужно также и отфильтровать неправдоподобное (следующий номер на 1 больше предыдущего) и заполнить пропуски

In [7]:
def get_text_value(v):
    return v if v and v is not np.nan else ''

def get_int_value(v):
    try:
        return int(v)
    except:
        return None

def get_page_numbers(df, start_page_number = 1):
    row_values_concatenation = df[df.columns[0]].apply(get_text_value)
    for col in df.columns[1:]:
        row_values_concatenation += df[col].apply(get_text_value)
    row_int_values_concatenation = row_values_concatenation.apply(get_int_value)
    page_numbers = row_int_values_concatenation.copy()

    cur_page = start_page_number
    for i in page_numbers.keys():
        if np.isnan(page_numbers[i]) or page_numbers[i]<cur_page or page_numbers[i]>cur_page+10:
            page_numbers[i]=cur_page
        else:
            cur_page = page_numbers[i]
    
    return (page_numbers+1).astype(int), row_int_values_concatenation

In [8]:
fr1['page'], rn1 = get_page_numbers(fr1)

In [9]:
fr2['page'], rn2 = get_page_numbers(fr2, start_page_number=8279)

### Выделим номера приложений по страницам

In [10]:
def get_doc_part(page_num):
    PAGE_STARTS = [1, 602, 1263, 10244, 10390]
    for i, ps in enumerate(PAGE_STARTS[1:]):
        if page_num < ps:
            return i+1
    return len(PAGE_STARTS)

In [11]:
fr1['part'] = fr1.page.apply(get_doc_part)
fr2['part'] = fr2.page.apply(get_doc_part)

### Удалим лишние столбцы

In [12]:
cleared_fr1 = fr1[list(fr1.columns[:3])+list(fr1.columns[-2:])]
cleared_fr1.columns = ['fio', 'comment', 'resolution', 'page', 'part']

cleared_fr2 = fr2[list(fr2.columns[:3])+list(fr2.columns[-2:])]
cleared_fr2.columns = ['fio', 'comment', 'resolution', 'page', 'part']

df = cleared_fr1.append(cleared_fr2)
df.index = range(len(df))

### Удалим пустые строчки и сторчки с номерами страниц

In [13]:
not_empty_strings = ~(df.fio.isnull()&df.comment.isnull()&df.resolution.isnull())
rows_not_just_page_numbers = rn1.append(rn2).isnull()
rows_not_just_page_numbers.index = range(len(df))
df = df[not_empty_strings&rows_not_just_page_numbers]
df.index = range(len(df))

### Размечаем подходящие на первый взгляд ФИО

In [14]:
from string import punctuation
regex_extra_spaces = re.compile('\s+')
regex_punctuation = re.compile('[%s]' % re.escape(punctuation))

def remove_punctuation_and_extra_spaces(t):
    t = str(t)
    t = regex_punctuation.sub(' ', t)
    t = regex_extra_spaces.sub(' ', t)
    t = t.strip()
    return t

In [15]:
fio_cap = '[А-ЯЁ][а-яё]+'
fio_space = '\s+'
fio_i = '[А-ЯЁа-яёA-Za-z][\.|\,]?'
fio_ii = fio_i+fio_space+fio_i
fio_fio_1part = '[А-ЯЁ][а-яё]+\s?(?:-|–)\s?' # двойная фамилия

fio_patterns = [
    fio_fio_1part+fio_cap+fio_space+fio_cap+fio_space+fio_cap,
    fio_fio_1part+fio_cap+fio_space+fio_cap,
    fio_fio_1part+fio_cap+fio_space+fio_ii,
    fio_fio_1part+fio_cap+fio_space+fio_i,

    fio_cap+fio_space+fio_cap+fio_space+fio_cap,
    fio_cap+fio_space+fio_cap,
    fio_cap+fio_space+fio_ii,
    fio_cap+fio_space+fio_i,

    fio_ii+fio_space+fio_cap,
    
    fio_cap+fio_space+fio_i,
    fio_cap
]
fio_pattern = '|'.join(['('+i+')\\b' for i in fio_patterns])
fio_regex = re.compile(fio_pattern)

In [16]:
def matches_fio_pattern(t):
    matches_fio = fio_regex.sub('', remove_punctuation_and_extra_spaces(t)).strip() == ''
    colletive_thing = '(коллективн' in str(t) and len(t)<100 and len(fio_regex.findall(t))>0
    return matches_fio or colletive_thing

In [17]:
df['matches_fio'] = df.fio.apply(matches_fio_pattern)

In [18]:
df.matches_fio.value_counts()

False    152068
True     106979
Name: matches_fio, dtype: int64

--------

# Сохраняем жадный результат: оставляем только валидные строчки

In [19]:
df_res = df[df.matches_fio&((~df.comment.isnull())&(~df.resolution.isnull()))]
df_res.index = range(len(df_res))

In [20]:
df_res = df_res[['part','page','fio','comment','resolution']]

In [21]:
df_res.head()

,part,page,fio,comment,resolution
0,1,9,Купцов Л.А.,Учитывая социальную значимость проекта строите...,Предложение не рекомендовано к учёту. Отсутств...
1,1,9,Макшанцев М.Н.,Установить зону развития по адресу: Новодевичи...,Предложение не рекомендовано к учёту. Отсутств...
2,1,9,Фомина Г.В.,С проектом ПЗЗ ознакомлена,Принято к сведению
3,1,9,Чернышов В.Н.,"С проектом ПЗЗ ознакомлен. Все хорошо, предлож...",Принято к сведению
4,1,9,Мешкова Н.С.,"С проектом ПЗЗ ознакомлена, предложений замеча...",Принято к сведению


In [22]:
df_res.shape

(51817, 5)

In [23]:
df_res.to_excel('data_progress/1 PZZ.xlsx')

----